# Bilden eines Models mit Original und einem Filter

## Import der notwendigen Pakete

In [1]:
# Import Tensorflow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## GPU-Test

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available", len(physical_devices))

Num GPUs Available 1


## Helper

In [3]:
model_path = "./keras_alle_filter.hdf5"
import os.path

def isModelAvailable():
    return os.path.isfile(model_path) 

startTraining = True

## Import des Datasets

Zusätzlich zum Laden des Datasets (23 Klassen) wird hier bereits durch den ImageDataGenerator ein Preprocessing durchgeführt / vorbereitet.
Zunächst wird versucht eine Klassifizierung durchzuführen, bei dem die Bilder in Graustufen geladen werden.

In [4]:
# path to dataset
directory = "./FACD_image"
target_size = (224,224)
seed = 42;
batch_size=32

# create a image generator for keras, that can load images batchwise
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
    validation_split=0.2
)

train_iterator = tf.keras.preprocessing.image.DirectoryIterator(
    directory, data_generator, target_size=target_size, color_mode='rgb', class_mode='categorical', 
    batch_size=batch_size, shuffle=True, seed=seed,
    follow_links=False, subset='training', interpolation='nearest', dtype=None
)

test_iterator = tf.keras.preprocessing.image.DirectoryIterator(
    directory, data_generator, target_size=target_size, color_mode='rgb',
    class_mode='categorical', batch_size=batch_size, shuffle=True, seed=seed,
    follow_links=False, subset='validation', interpolation='nearest', dtype=None
)

Found 23552 images belonging to 23 classes.
Found 5888 images belonging to 23 classes.


## Erstellen des CNN-Models

In [5]:
input_shape = (224,224,3)
num_classes = 23

metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]

model = tf.keras.Sequential()
model.add(keras.Input(shape=input_shape))
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.25))
model.add(layers.Dense(units=128))
model.add(layers.Dense(units=84))
model.add(layers.Dense(num_classes, activation="softmax"))
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), 
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=metrics)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 32)      9248      
_________________________________________________________________
dropout (Dropout)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 64)        3

## Trainieren des Models

Das Warning "This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling" kommt von den nicht gefitteten Bildern des DataGenerators. Normalerweise führt man "data_generator.fit()" vor dem fitten des Modells durch. In der [Doku des DictornaryIterators](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/DirectoryIterator) steht jedoch, dass das durch das übergebene ImageDateGenerator-Objekt eine Standardisierung und Transformation durchgeführt wird. Aus diesem Grund wird dieses Warning an dieser Stelle ignoriert.

In [6]:
epochs = 20
validation_freq = 1
if not isModelAvailable() or startTraining:
    
    es = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
        min_delta=1e-2,
        patience=2,
        verbose=1
    )
    
    mc = tf.keras.callbacks.ModelCheckpoint(
        "model_all_{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_loss', verbose=0, save_best_only=True,
        save_weights_only=False, mode='auto', save_freq='epoch'
    )
    
    history = model.fit(
        x=train_iterator, y=None, epochs=epochs, verbose=1, callbacks=[es, mc],
        validation_data=test_iterator, initial_epoch=0, steps_per_epoch=len(train_iterator), validation_freq=validation_freq,
        max_queue_size=10, workers=4
    )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 736 steps, validate for 184 steps
Epoch 1/20
652/736 [=========================>....] - ETA: 9s - loss: 9.7805 - categorical_accuracy: 0.0409 - precision: 0.0503 - recall: 4.7929e-04

C:\Users\Martin\anaconda3\envs\ml\lib\site-packages\PIL\TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19273 bytes but only got 816. Skipping tag 700
  " Skipping tag %s" % (size, len(data), tag)
C:\Users\Martin\anaconda3\envs\ml\lib\site-packages\PIL\TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5140 bytes but only got 816. Skipping tag 34377
  " Skipping tag %s" % (size, len(data), tag)
C:\Users\Martin\anaconda3\envs\ml\lib\site-packages\PIL\TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3144 bytes but only got 816. Skipping tag 34675
  " Skipping tag %s" % (size, len(data), tag)


735/736 [============================>.] - ETA: 0s - loss: 9.0302 - categorical_accuracy: 0.0416 - precision: 0.0503 - recall: 4.2517e-04WARNING:tensorflow:From C:\Users\Martin\anaconda3\envs\ml\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1786: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model_all_01-3.14\assets
736/736 [==============================] - 138s 188ms/step - loss: 9.0222 - categorical_accuracy: 0.0415 - precision: 0.0503 - recall: 4.2459e-04 - val_loss: 3.1364 - val_categorical_accuracy: 0.0479 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/20
736/736 [==============================] - 137s 186ms/step - loss: 3.1276 - categorical_accuracy: 0.0484 - precision: 0.8971 - recall: 0.0026 - val_loss: 3.1100 - v

KeyError: 'val_loss'

## Speichern des Modells

In [ ]:
if not isModelAvailable():
    model.save(
       model_path, overwrite=True, include_optimizer=True, save_format="h5", signatures=None, options=None
    )

## Anzeigen der Lern- und Fehlerkurve

In [ ]:
import importlib
import plot #import the module here, so that it can be reloaded.
importlib.reload(plot)

plot.plot_keras_history(history, 11, metrics[0].name)

In [ ]:
plot.plot_keras_history(history, 11, metrics[1].name)

In [ ]:
plot.plot_keras_history(history, 11, metrics[2].name)